In [13]:
import PyPDF2
import openai
from dotenv import load_dotenv

load_dotenv()

True

# Creation of the gpt

In [60]:
client = openai.Client()
model = "gpt-3.5-turbo"
main_prompt = """You will convert text data into a structured CSV format. The text data comes from a swimming competition result sheet and includes columns like 'primer apellido' (first surname), 'segundo apellido' (second surname), 'nombre' (name), 'año' (year), 'género' (gender), 'equipo' (team), 'puntos' (points), 'tiempo' (time), 'estilo' (style), 'distancia' (distance), 'tiempo 50m', and so on. The data is separated by commas.

Here's an example of the text data:
'Splash Meet Manager, ... Prueba 13 Masc., 200m Libre Absoluto Masculino 25/02/2023 - 11:15 Resultados Clasificación AN Tiempo Puntos 50m 100m 150m 200m 1.GUTIERREZ RAMOS, R. 04 C.D.N. Ciudad De Algeciras 2:01.35 19,00 27.26 30.06 31.70 32.33 2.GALLARDO MARTIN, J. 04 C.D.N. Inacua Malaga 2:04.57 16,00 28.69 30.89 32.15 32.84...'

You need to extract and reformat this data into a CSV format like this:
'primer apellido, segundo apellido, nombre, año, género, equipo, puntos, tiempo, estilo, distancia, tiempo 50m, tiempo 100m, tiempo 150m, tiempo 200m, tiempo 250m, tiempo 300m, tiempo 350m, tiempo 400m, tiempo 800m, tiempo 1500m, fecha, hora, lugar
GUTIERREZ, RAMOS, R., 04, Masc, C.D.N. Ciudad De Algeciras, 19, 2:01.35, Libre, 200m, 27.26, 30.06, 31.70, 32.33, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras
GALLARDO, MARTIN, J., 04, Masc, C.D.N. Inacua Malaga, 16, 2:04.57, Libre, 200m, 28.69, 30.89, 32.15, 32.84, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras'

Note: If a swimmer is disqualified, indicated by 'WDR', then the time should be 'None' and the points should be '0'.

When extracting data, ensure each row has the same number of fields. If the last row or any other row has missing information, fill in those fields with 'None'. Pay special attention to the last row to maintain consistency with the rest of the data.


"""

messages = [
    {"role": "system", "content": main_prompt}
]
#Append the message to the conversation history 
def add_message(role, message):
    messages.append({"role": role, "content": message})
    return messages
def converse_with_chatGPT(messages):
    completion = client.chat.completions.create(
    model=model, #Open AI model name
    messages=messages, # user query
    n=1, #number of responses expected from the Chat GPT
    temperature=0 #making responses deterministic
)

    return completion.choices
# process user prompt
def process_user_query(prompt, messages):
    user_prompt = (f"{prompt}")
    add_message("user", user_prompt)
    result = converse_with_chatGPT(messages)
    return result
#Request user to provide the query
def user_query(prompt):
    prompt = prompt
    response = process_user_query(prompt, messages)
    return response[0].message.content
        

# First try

In [61]:
pdf = PyPDF2.PdfReader("ResultList_13.pdf")
# get number of pages
text = ""
num_pages = len(pdf.pages)
for page in range(num_pages):
    pageObj = pdf.pages[page]
    text += pageObj.extract_text()
print(text)

Splash Meet Manager, 11.75980 Registered to Andalucía 25/02/2023 11:23 - Página 1
XXVII TROFEO NATACIÓN  "DÍA DE ANDALUCÍA"
ALGECIRAS, 25/2/2023
Prueba 13 Masc., 200m Libre Absoluto Masculino
25/02/2023 - 11:15 Resultados
Clasificación AN Tiempo Puntos 50m 100m 150m 200m
1.GUTIERREZ RAMOS, R. 04 C.D.N. Ciudad De Algeciras 2:01.35 19,00 27.26 30.06 31.70 32.33
2.GALLARDO MARTIN, J. 04 C.D.N. Inacua Malaga 2:04.57 16,00 28.69 30.89 32.15 32.84
3.KENT, Merrick 05 Tarik Waterpolo Team 2:04.98 14,00 28.52 31.55 32.92 31.99
4.ROBLES ROMERO, Javier 06 C.D.N. Sanlucar 2:09.94 13,00 28.43 32.66 34.52 34.33
5.MACLANDA BUSTO, J. 03 C.D.N. Ciudad De Algeciras 2:12.57 - 29.77 32.64 35.37 34.79
6.GAITÁN CANTOS, Ignacio 99 C.N. Caballa - Ciudad De Ceuta 2:17.47 12,00 31.29 35.61 36.39 34.18
7.GARCÍA LUCERO, Jorge 09 C.N. Portuense 2:17.78 11,00 31.78 35.05 36.29 34.66
8.DOMÍNGUEZ RUIZ, Álvaro 09 C.N. Dos Hermanas 2:24.00 10,00 31.29 36.25 38.79 37.67
9.LANZA PEREA, Rafael 08 C.N. Torremolinos 2:24.60

In [62]:
datos = user_query(text)
print(datos)

primer apellido, segundo apellido, nombre, año, género, equipo, puntos, tiempo, estilo, distancia, tiempo 50m, tiempo 100m, tiempo 150m, tiempo 200m, tiempo 250m, tiempo 300m, tiempo 350m, tiempo 400m, tiempo 800m, tiempo 1500m, fecha, hora, lugar
GUTIERREZ, RAMOS, R., 04, Masc, C.D.N. Ciudad De Algeciras, 19, 2:01.35, Libre, 200m, 27.26, 30.06, 31.70, 32.33, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras
GALLARDO, MARTIN, J., 04, Masc, C.D.N. Inacua Malaga, 16, 2:04.57, Libre, 200m, 28.69, 30.89, 32.15, 32.84, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras
KENT, None, Merrick, 05, None, Tarik Waterpolo Team, 14, 2:04.98, Libre, 200m, 28.52, 31.55, 32.92, 31.99, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras
ROBLES, ROMERO, Javier, 06, None, C.D.N. Sanlucar, 13, 2:09.94, Libre, 200m, 28.43, 32.66, 34.52, 34.33, None, None, None, None, None, None, 25/02/2023, 11:15, Algeciras
MACLANDA, BUSTO, J., 03, None, C.D.N. Ciudad De Algeciras,

In [63]:
if datos is not None:
    with open('datos.csv', 'w') as f:
        f.write(datos)
else:
    print("No data to write to the file.")

In [64]:
import pandas as pd

csv = pd.read_csv("datos.csv")
print(csv)

   primer apellido  segundo apellido      nombre   año  género  \
0        GUTIERREZ             RAMOS          R.     4    Masc   
1         GALLARDO            MARTIN          J.     4    Masc   
2             KENT              None     Merrick     5    None   
3           ROBLES            ROMERO      Javier     6    None   
4         MACLANDA             BUSTO          J.     3    None   
5           GAITÁN            CANTOS     Ignacio    99    None   
6           GARCÍA            LUCERO       Jorge     9    None   
7        DOMÍNGUEZ              RUIZ      Álvaro     9    None   
8            LANZA             PEREA      Rafael     8    None   
9          SÁNCHEZ              LOBO      Javier     7    None   
10         JARILLO            PINEDA      Adrian     7    None   
11         SEQUERA             REINA      Carlos     7    None   
12           VERDU            MATEOS   Jose Luis     9    None   
13         ATENCIA          GONZALEZ          G.     6    None   

         

# Second try

In [65]:
pdf = PyPDF2.PdfReader("ResultList_15.pdf")
# get number of pages
text = ""
num_pages = len(pdf.pages)
for page in range(num_pages):
    pageObj = pdf.pages[page]
    text += pageObj.extract_text()
print(text)

Splash Meet Manager, 11.76277 Registered to Andalucía 01/04/2023 12:58 - Página 1
FASE 2 RFEN ALEVÍN ZONA CENTRAL
LOS BARRIOS, 1/4/2023
Prueba 15 Masc., 400m Libre Alevín Masc.
01/04/2023 - 12:51 Resultados
Clasificación AN Tiempo Puntos
1.GIL DE MIGUEL, Nicolas 09 C.N. Los Barrios 4:53.58 38,00
50m: 33.60 33.60 150m: 1:45.57 36.48 250m: 3:01.37 38.46 350m: 4:17.69 37.70
100m: 1:09.09 35.49 200m: 2:22.91 37.34 300m: 3:39.99 38.62 400m: 4:53.58 35.89
2.SANTANO JURADO, Francisco Javier 09 C.N. Baena 4:53.69 32,00
50m: 33.72 33.72 150m: 1:49.94 38.32 250m: 3:06.12 37.91 350m: 4:19.85 36.08
100m: 1:11.62 37.90 200m: 2:28.21 38.27 300m: 3:43.77 37.65 400m: 4:53.69 33.84
3.BRICEÑO CORTES, Iker 10 C.D.N. Inacua Malaga 4:54.03 28,00
50m: 33.88 33.88 150m: 1:48.60 37.97 250m: 3:04.61 38.19 350m: 4:19.81 37.18
100m: 1:10.63 36.75 200m: 2:26.42 37.82 300m: 3:42.63 38.02 400m: 4:54.03 34.22
4.GONZALEZ DÍAZ, Javier 09 C.N. Fuengirola 4:55.53 26,00
50m: 34.86 34.86 150m: 1:48.56 36.64 250m: 3:03.77 

In [66]:
datos = user_query(text)
print(datos)

primer apellido, segundo apellido, nombre, año, género, equipo, puntos, tiempo, estilo, distancia, tiempo 50m, tiempo 100m, tiempo 150m, tiempo 200m, tiempo 250m, tiempo 300m, tiempo 350m, tiempo 400m, tiempo 800m, tiempo 1500m, fecha, hora, lugar
GIL, DE MIGUEL, Nicolas, 09, Masc, C.N. Los Barrios, 38, 4:53.58, Libre, 400m, 33.60, 1:09.09, 1:45.57, 2:22.91, 3:01.37, 3:39.99, 4:17.69, 4:53.58, None, None, 01/04/2023, 12:51, Los Barrios
SANTANO, JURADO, Francisco Javier, 09, Masc, C.N. Baena, 32, 4:53.69, Libre, 400m, 33.72, 1:11.62, 1:49.94, 2:28.21, 3:06.12, 3:43.77, 4:19.85, 4:53.69, None, None, 01/04/2023, 12:51, Los Barrios
BRICEÑO, CORTES, Iker, 10, Masc, C.D.N. Inacua Malaga, 28, 4:54.03, Libre, 400m, 33.88, 1:10.63, 1:48.60, 2:26.42, 3:04.61, 3:42.63, 4:19.81, 4:54.03, None, None, 01/04/2023, 12:51, Los Barrios
GONZALEZ, DÍAZ, Javier, 09, Masc, C.N. Fuengirola, 26, 4:55.53, Libre, 400m, 34.86, 1:11.92, 1:48.56, 2:25.40, 3:03.77, 3:41.99, 4:19.46, 4:55.53, None, None, 01/04/2023,

In [67]:
with open('datos2.csv', 'w') as f:
    f.write(datos)

In [68]:
import pandas as pd
csv2 = pd.read_csv("datos2.csv")
print(csv2)

  primer apellido  segundo apellido             nombre   año  género  \
0             GIL         DE MIGUEL            Nicolas     9    Masc   
1         SANTANO            JURADO   Francisco Javier     9    Masc   
2         BRICEÑO            CORTES               Iker    10    Masc   
3        GONZALEZ              DÍAZ             Javier     9    Masc   
4           PAVÓN          WEITZNER                Max     9    Masc   

                  equipo   puntos    tiempo  estilo  distancia  ...  \
0       C.N. Los Barrios       38   4:53.58   Libre       400m  ...   
1             C.N. Baena       32   4:53.69   Libre       400m  ...   
2   C.D.N. Inacua Malaga       28   4:54.03   Libre       400m  ...   
3        C.N. Fuengirola       26   4:55.53   Libre       400m  ...   
4        C.N. Fuengirola       24   4:57.49   Libre       400m  ...   

    tiempo 200m  tiempo 250m  tiempo 300m  tiempo 350m  tiempo 400m  \
0       2:22.91      3:01.37      3:39.99      4:17.69      4:53.58  